In [1]:
import re

import requests
import bs2json as bs2
import rolling_pin.blob_etl as rpb

In [2]:
def url_to_blob(url):
    html = requests.get(url).content.decode('utf-8')
    data = bs2.BS2Json(html).convert()
    blob = rpb.BlobETL(data)
    return blob

def url_to_networkx(url):
    return url_to_blob(url).to_networkx_graph()

In [3]:
url = 'https://github.com/theNewFlesh/rolling-pin'
graph = url_to_networkx(url)
graph

In [23]:
blob = url_to_blob(url)

func = lambda x: re.sub(':', '_', str(x))
r = blob \
    .set(
        key_setter=lambda k, v: func(k),
        value_setter=lambda k, v: func(v),
    )

q = 'docker'
r \
    .filter(lambda x: bool(re.search(q, x)), by='value') \
    .to_html()

In [28]:
blob = url_to_blob(url)

func = lambda x: re.sub(':', '_', str(x))
r = blob.set(
    key_setter=lambda k, v: func(k),
    value_setter=lambda k, v: func(v),
)

r \
    .query('/p/') \
    .filter(lambda x: bool(re.search('python', x, re.I)), by='value') \
    .to_html()

In [39]:
blob = url_to_blob(url)

func = lambda x: re.sub(':', '_', str(x))
r = blob.set(
    key_setter=lambda k, v: func(k),
    value_setter=lambda k, v: func(v),
)

sep = '}'
r = r \
    .query('/p/') \
    .filter(lambda x: bool(re.search('python', x, re.I)), by='value') \
    .set(key_setter=lambda k, v: re.sub('(.*?)/(.*)', f'{sep}\\1{sep}\\2', k)) \
    .set(key_setter=lambda k, v: re.sub('(.*)/(.*)/(.*)', f'\\1{sep}\\2{sep}\\3', k)) \
    .to_flat_dict()
r = rpb.BlobETL(r, separator=sep)
r.to_html(orient='lr')